In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# 크롤링할 값을 변수로 설정
query = "OpenAI"  # 검색어
start_date = "2025.02.01"  # 시작 날짜
end_date = "2025.03.31"  # 종료 날짜
max_news = 20  # 최대 뉴스 개수
news_per_page = 10  # 네이버 뉴스 검색 결과 한 페이지당 뉴스 개수

# 뉴스 데이터 저장용 리스트
data = []

# 여러 페이지에서 크롤링하기
for start in range(1, max_news, news_per_page):
    # URL 생성 (페이지네이션 반영)
    url = f"https://search.naver.com/search.naver?where=news&query={query}&ds={start_date}&de={end_date}&sort=0&start={start}"

    # 요청 보내기
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # 뉴스 리스트 추출
    news_list = soup.select("ul.list_news > li.bx")

    # 뉴스 데이터 저장
    for news in news_list:
        # 기사 제목과 URL
        title_tag = news.select_one("a.news_tit")
        if not title_tag:
            continue  # 제목이 없으면 스킵
        title = title_tag.get_text()
        article_url = title_tag['href']

        # 기사 요약 내용
        summary_tag = news.select_one("a.api_txt_lines.dsc_txt_wrap")
        summary = summary_tag.get_text() if summary_tag else "요약 없음"

        # 언론사 이름
        press_tag = news.select_one("a.info.press")
        press_name = press_tag.get_text() if press_tag else "언론사 없음"

        # 기사 날짜
        date_tag = news.select("span.info")
        if date_tag:
            relative_date = date_tag[0].get_text()
            current_date = datetime.now().strftime("%Y.%m.%d")
            date = f"{current_date} ({relative_date})"
        else:
            date = "날짜 정보 없음"

        # 데이터 저장
        data.append({
            'title': title,
            'url': article_url,
            'summary': summary,
            'press': press_name,
            'date': date
        })

    # 만약 설정한 max_news 개수 이상이면 종료
    if len(data) >= max_news:
        break

# 결과 출력
for item in data:
    print(f"제목: {item['title']}")
    print(f"언론사: {item['press']}")
    print(f"날짜: {item['date']}")
    print(f"URL: {item['url']}")
    print(f"요약: {item['summary']}")
    print("-" * 400)


In [ ]:
pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

options = Options()
options.add_argument('--headless')  # 창 없이 실행
options.add_argument('--disable-gpu')
driver = webdriver.Chrome(options=options)

query = "OpenAI"
start_date = "2025.02.01"
end_date = "2025.03.31"
max_news = 20
news_per_page = 10
data = []

for start in range(1, max_news, news_per_page):
    url = f"https://search.naver.com/search.naver?where=news&query={query}&ds={start_date}&de={end_date}&sort=0&start={start}"
    driver.get(url)
    time.sleep(2)  # JS 렌더링 대기

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_list = soup.select("ul.list_news > li.bx")

    for news in news_list:
        title_tag = news.select_one("a.news_tit")
        if not title_tag:
            continue
        title = title_tag.get_text()
        article_url = title_tag['href']
        summary_tag = news.select_one("a.api_txt_lines.dsc_txt_wrap")
        summary = summary_tag.get_text() if summary_tag else "요약 없음"
        press_tag = news.select_one("a.info.press")
        press_name = press_tag.get_text() if press_tag else "언론사 없음"
        date_tag = news.select("span.info")
        date = date_tag[0].get_text() if date_tag else "날짜 정보 없음"

        data.append({
            'title': title,
            'url': article_url,
            'summary': summary,
            'press': press_name,
            'date': date
        })

    if len(data) >= max_news:
        break

driver.quit()

# 결과 출력
for item in data:
    print(f"제목: {item['title']}")
    print(f"언론사: {item['press']}")
    print(f"날짜: {item['date']}")
    print(f"URL: {item['url']}")
    print(f"요약: {item['summary']}")
    print("-" * 400)


In [ ]:

import random

# 기사 목록 출력
print("기사 목록:")
for idx, article in enumerate(data):
    print(f"{idx + 1}: {article['title']}")

# 사용자 입력으로 기사 선택
if len(data) >= 2:
    try:
        selected_indices = input("=====선택할 기사 번호 두 개를 입력하세요 (예: 1 3): ").split()
        selected_indices = [int(i) - 1 for i in selected_indices]

        if len(selected_indices) != 2 or any(i < 0 or i >= len(data) for i in selected_indices):
            raise ValueError("잘못된 입력입니다. 올바른 기사 번호 두 개를 입력하세요.")

        selected_articles = [data[selected_indices[0]], data[selected_indices[1]]]
        print("---"*50)
        print("선택된 기사 제목:")
        print("1:", selected_articles[0]['title'])
        print("2:", selected_articles[1]['title'])
    except ValueError as e:
        print(f"입력 오류: {e}")
else:
    print("기사가 충분하지 않습니다.")

In [ ]:
import random

# 기사 2개 랜덤 선별
if len(data) >= 2:
    selected_articles = random.sample(data, 2)
    print("선택된 기사 제목:")
    print("1:", selected_articles[0]['title'])
    print("2:", selected_articles[1]['title'])
else:
    print("기사가 충분하지 않습니다.")


In [ ]:
import os
from getpass import getpass

# 사용자에게 API 키를 입력받아 환경변수에 저장
os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

prompt = f"""
아래의 두 기사를 바탕으로 '제목, 서론, 본론, 결론'의 형식을 갖춘 하나의 자연스러운 뉴스 기사로 작성해주세요. 서론:, 본론:, 결론: 이라고 적지말고 그냥 한 개의 글로 적어주세요.
제목도 최적화 검색 엔진을 통해 사람들이 많이 클릭할 수 있도록 해줘.
각 구성 요소는 명확히 구분하고, 줄바꿈을 통해 가독성을 높여주세요.
또한 실제 기사처럼 글자수를 1000자~2000자 사이로 맞추고 내용을 풍부하게 적어주세요.
마지막으로, 이 기사를 읽고 느낀 것을 다양한 관점에서 적어주세요.

기사 1:
{selected_articles[0]['summary']}

기사 2:
{selected_articles[1]['summary']}

새로 작성된 기사:
"""


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=2500,
    temperature=0.7
)

revised_article = response.choices[0].message.content.strip()
print("[재편집된 기사]")
print(revised_article)


In [ ]:
!pip install requests PyJWT

In [ ]:
!pip install PyJWT requests

In [ ]:
# main prompt
import os
from openai import OpenAI
import ipywidgets as widgets
from IPython.display import display

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# 프롬프트 설정
prompt = f"""
아래의 두 기사의 내용을 모두 담아서, '제목, 서론, 본론, 결론'의 형식을 갖춘 하나의 자연스러운 뉴스 기사를 작성해주세요.

아래 사항을 반드시 지켜주세요:

1. 반드시 기사의 맨 첫 줄에 [] 안에 매력적이고 클릭하고 싶어질 만한 제목을 작성해주세요.
2. 제목과 본문 내용을 검색 엔진 최적화(SEO)를 고려해서 작성해주세요.
3. 서론, 본론, 결론이라고 명시하지 말고, 자연스럽게 누군가와 대화하는 듯한 느낌으로 글을 작성해주세요. 모르는 사람에게 알려주듯이 친근하게 말하는 방법도 좋습니다.
4. 각 구성 요소는 줄바꿈을 통해 명확히 구분하여 가독성을 높여주세요.
5. 실제로 누군가와 대화하는 것처럼 글자 수를 1200자~1600자 사이로 맞춰 풍부한 내용을 담아주세요.
6. 대화를 통해 누군가에게 설명해주듯이 말하면서 중간중간 자문자답 형식 1~2번 포함되면 정말 좋을 거 같아요. (필수로 질문 문장에서는 꼭 줄바꿈을 해줘.)
7. 2개의 기사 내용이 골고루 들어가하고 기사의 내용에 기반하여 사실만 전달해주세요.

기사 1:
{selected_articles[0]['summary']}

기사 2:
{selected_articles[1]['summary']}

새로 작성된 기사:
"""

# GPT-4o를 사용하여 기사 생성
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=3000,
    temperature=0.7
)

# 기사 초안 저장
revised_article = response.choices[0].message.content.strip()

# Colab에서 직접 수정할 수 있는 입력창 생성
text_area = widgets.Textarea(
    value=revised_article,
    placeholder="여기에 기사를 수정하세요...",
    layout=widgets.Layout(width="100%", height="400px")
)

# "확정" 버튼 추가
confirm_button = widgets.Button(description="기사 확정")

# 확정된 기사를 저장할 변수
final_article = ""

# 버튼 클릭 이벤트 핸들러
def on_confirm_clicked(b):
    global final_article
    final_article = text_area.value
    print("\n 최종 확정된 기사:\n")
    print(final_article)

confirm_button.on_click(on_confirm_clicked)

# UI 표시
display(text_area, confirm_button)
